In [2]:
import sys
sys.path.insert(1, "./")

from algorithms import *

import pandas as pd
import numpy as np
import copy
import itertools
import os
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.optimize import linprog
import math
import scipy.stats
from scipy.spatial import KDTree
import tqdm
import re

An example definition of the set system:

In [ ]:
n = 5 # Size of ground set U = {0, 1, 2, ..., n}
sets = [
    {0, 1},
    {0, 2},
    {0, 3},
    {0, 4},
    {0, 4, 1},
    {0, 1}
]
colors = [0, 0, 1, 1, 1, 1] # 0 --> red, 1 --> blue
m = len(sets) # Number of sets

However, we need to read from a dataset and preprocess and create the variables `sets`, `colors`, etc from the csv file.

Example on Adults dataset:

In [7]:
data = pd.read_csv("dataset/adult.csv")

data.shape, data["gender"].value_counts()

((48842, 15),
 gender
 Male      32650
 Female    16192
 Name: count, dtype: int64)

Sample a speicfic amount of data to work on:

In [8]:
df = data.sample(n=4000)

Now, we define the columns that we want to cover, by choosing a subset of rows. 

For example, we want our subset (of rows) to cover both `<=50k` and `>50k` values from the column `income`.

In [10]:
cover_columns = {
    "workclass": ["Private", "Self-emp-not-inc", "Local-gov", "State-gov", "Self-emp-inc", "Federal-gov"], 
    "education": ["HS-grad", "Some-college", "Bachelors", "Masters", "Assoc-voc", "11th", "Assoc-acdm", "10th", "7th-8th"], 
    "marital-status": ["Married-civ-spouse", "Never-married", "Divorced", "Widowed"], 
    "occupation": ["Craft-repair", "Adm-clerical", "Prof-specialty", "Exec-managerial", "Sales", "Other-service", "Machine-op-inspct", "Transport-moving"], 
    "income": ["<=50K", ">50K"]
}

In the following step, we create the `sets` and `colors` variables to pass to our implemented algorithms.

In [11]:
# Each column value acts like a points in the set systems
points = []
for k in cover_columns.keys():
    points += cover_columns[k]
print(len(points))

29


**Note:** We use the index of a point (value) to specify a point in our point system. In other words, `X = [0, 1, ..., len(points) - 1]`.

Here `X` is the set of points.

In [13]:
# Each row is a set in the set system
def make_sets(df, cover_columns, points):
    sets = []
    colors = []
    for _, row in df.iterrows():
        this_set = []
        for col in cover_columns.keys():
            if row[col] in points:
                this_set.append(points.index(row[col]))
        sets.append(set(this_set))
        colors.append(1 if row["gender"] == "Male" else 0)
    return sets, colors

# The variables 'sets' and 'colors'
sets, colors = make_sets(df, cover_columns, points)
len(sets), len(colors)

(4000, 4000)

Look at the demographic groups distribution in this sample:

In [14]:
np.unique(np.array(colors), return_counts=True), df["gender"].value_counts()

((array([0, 1]), array([1308, 2692])),
 gender
 Male      2692
 Female    1308
 Name: count, dtype: int64)

## Run & Report

In [ ]:
def run_with_time(func, *args):
    start_time = time.time()
    res = func(*args)
    run_time = time.time() - start_time
    return (res, run_time)

def run_all(n, sets, colors):
    results = {}

    print("Running SC...")
    results["SC"] = run_with_time(standard_set_cover, n, copy.deepcopy(sets))
    print("Running Naive...")
    results["Naive"] = run_with_time(naive_fair_set_cover, n, copy.deepcopy(sets), copy.deepcopy(colors))
    print("Running AllPick...")
    results["AllPick"] = run_with_time(all_pick_fair_set_cover, n, copy.deepcopy(sets), copy.deepcopy(colors))
    print("Running EffAllPick...")
    results["EffAllPick"] = run_with_time(efficient_pick_set_cover, n, copy.deepcopy(sets), copy.deepcopy(colors))

    return results

In [ ]:
RUN_PER_N = 5 # number of runs per `N`
N = len(points)

# Different set sizes. This will be used as the sample size from the Dataframe `data`.
set_sizes = [1000, 2000, 3000, 4000, 5000]

result_df_times = {"SetSize": [], "SC": [], "Naive": [], "AllPick": [], "EffAllPick": []}
result_df_counts = {"SetSize": [], "SC": [], "Naive": [], "AllPick": [], "EffAllPick": []}
result_df_fairness = {"SetSize": [], "SC": [], "Naive": [], "AllPick": [], "EffAllPick": []}

for set_size in set_sizes:
    for i in range(RUN_PER_N):
        print("Preparing the sets")
        
        df = data.sample(n=set_size)
        sets, colors = make_sets(df, cover_columns, points)

        print("Running...")
        
        # Run
        result = run_all(N, sets, colors)

        print(f"set_size = {set_size} in {set_sizes} : {i + 1} / {RUN_PER_N}")

        # Generate Result Dataframe
        result_df_times["SetSize"].append(set_size)
        result_df_counts["SetSize"].append(set_size)
        result_df_fairness["SetSize"].append(set_size)
        for key in result:
            result_df_times[key].append(result[key][1])
            result_df_counts[key].append(len(result[key][0]))
            tmp = np.unique([colors[c] for c in result[key][0]], return_counts=True)[1]
            if len(tmp) == 1:
                result_df_fairness[key].append(0)
            else:
                result_df_fairness[key].append(np.min(tmp) / np.max(tmp))


result_df_times = pd.DataFrame(result_df_times)
result_df_counts = pd.DataFrame(result_df_counts)
result_df_fairness = pd.DataFrame(result_df_fairness)